# TaylorMap Demo

## 1. Initialize `mtflib` global parameters

This is a mandatory step before using any `mtflib` functionality. We set the maximum order of the Taylor series and the number of variables.

In [1]:
import numpy as np

from mtflib import TaylorMap, mtf

try:
    mtf.initialize_mtf(max_order=4, max_dimension=2)
except RuntimeError:
    pass

Initializing MTF globals with: _MAX_ORDER=4, _MAX_DIMENSION=2
Loading/Precomputing Taylor coefficients up to order 4
Global precomputed coefficients loading/generation complete.
Size of precomputed_coefficients dictionary in memory: 464 bytes, 0.45 KB, 0.00 MB
MTF globals initialized: _MAX_ORDER=4, _MAX_DIMENSION=2, _INITIALIZED=True
Max coefficient count (order=4, nvars=2): 15
Precomputed coefficients loaded and ready for use.


## 2. Create two `TaylorMap` objects

Let's define two maps from R^2 to R^2.

### Map 1: F(x,y) = [sin(x), cos(y)]

In [2]:
x = mtf.var(1)
y = mtf.var(2)
sin_x = mtf.sin(x)
cos_y = mtf.cos(y)
map_F = TaylorMap([sin_x, cos_y])
print(map_F)

TaylorMap with 2 components (input dim: 2):
--- Component 1 ---
          Coefficient  Order Exponents
0  1.000000000000e+00      1    (1, 0)
1 -1.666666666667e-01      3    (3, 0)

--- Component 2 ---
          Coefficient  Order Exponents
0  1.000000000000e+00      0    (0, 0)
1 -5.000000000000e-01      2    (0, 2)
2  4.166666666667e-02      4    (0, 4)




### Map 2: G(x,y) = [x + y, x - y]

In [3]:
map_G = TaylorMap([x + y, x - y])
print(map_G)

TaylorMap with 2 components (input dim: 2):
--- Component 1 ---
         Coefficient  Order Exponents
0 1.000000000000e+00      1    (1, 0)
1 1.000000000000e+00      1    (0, 1)

--- Component 2 ---
          Coefficient  Order Exponents
0  1.000000000000e+00      1    (1, 0)
1 -1.000000000000e+00      1    (0, 1)




## 3. Demonstrate Operations

### Operation 1: Addition (F + G)

In [4]:
map_sum = map_F + map_G
print(map_sum)

TaylorMap with 2 components (input dim: 2):
--- Component 1 ---
          Coefficient  Order Exponents
0  2.000000000000e+00      1    (1, 0)
1  1.000000000000e+00      1    (0, 1)
2 -1.666666666667e-01      3    (3, 0)

--- Component 2 ---
          Coefficient  Order Exponents
0  1.000000000000e+00      0    (0, 0)
1  1.000000000000e+00      1    (1, 0)
2 -1.000000000000e+00      1    (0, 1)
3 -5.000000000000e-01      2    (0, 2)
4  4.166666666667e-02      4    (0, 4)




### Operation 2: Composition F(G(x,y))

This computes `sin(x+y)` and `cos(x-y)`.

In [5]:
map_composed = map_F.compose(map_G)
print(map_composed)

TaylorMap with 2 components (input dim: 2):
--- Component 1 ---
          Coefficient  Order Exponents
0  1.000000000000e+00      1    (1, 0)
1  1.000000000000e+00      1    (0, 1)
2 -1.666666666667e-01      3    (3, 0)
3 -5.000000000000e-01      3    (2, 1)
4 -5.000000000000e-01      3    (1, 2)
5 -1.666666666667e-01      3    (0, 3)

--- Component 2 ---
          Coefficient  Order Exponents
0  1.000000000000e+00      0    (0, 0)
1 -5.000000000000e-01      2    (2, 0)
2  1.000000000000e+00      2    (1, 1)
3 -5.000000000000e-01      2    (0, 2)
4  4.166666666667e-02      4    (4, 0)
5 -1.666666666667e-01      4    (3, 1)
6  2.500000000000e-01      4    (2, 2)
7 -1.666666666667e-01      4    (1, 3)
8  4.166666666667e-02      4    (0, 4)




### Operation 3: Trace

The trace is the sum of the diagonal elements of the Jacobian matrix's linear part. For `F(x,y) = [sin(x), cos(y)]`, the Jacobian is `[[cos(x), 0], [0, -sin(y)]]`. At (0,0), the linear part is `[[1, 0], [0, 0]]`, so the trace is 1.

In [6]:
trace_F = map_F.trace()
print(f"Trace of F at (0,0): {trace_F}")

Trace of F at (0,0): (1+0j)


### Operation 4: Substitution

Let's evaluate the composed map `F(G(x,y))` at `x=0.5, y=0.2`. This is equivalent to evaluating `[sin(x+y), cos(x-y)]` at the point, which is `sin(0.7)` and `cos(0.3)`.

In [7]:
eval_point = {1: 0.5, 2: 0.2}
result_array = map_composed.substitute(eval_point)
print(f"F(G(0.5, 0.2)) from TaylorMap: {result_array}")

# Compare with numpy to verify
numpy_result = [np.sin(0.7), np.cos(0.3)]
print(f"NumPy equivalent for comparison: {numpy_result}")

F(G(0.5, 0.2)) from TaylorMap: [0.64283333 0.9553375 ]
NumPy equivalent for comparison: [np.float64(0.644217687237691), np.float64(0.955336489125606)]


## 4. Demonstrate Map Inversion

### Operation 5: Inversion of a Map

Here we create an invertible map `F(x,y) = [x + 0.1*y^2, y - 0.1*x^2]`, invert it, and then compose the result with the original map to verify that we get the identity map.

In [8]:
# Create an invertible map
x_inv = mtf.var(1)
y_inv = mtf.var(2)
f1_inv = x_inv + 0.1 * y_inv**2
f2_inv = y_inv - 0.1 * x_inv**2
map_to_invert = TaylorMap([f1_inv, f2_inv])
print("--- Original Map to Invert ---")
print(map_to_invert)

# Invert the map
inverted_map = map_to_invert.invert()
print("\n--- Inverted Map ---")
print(inverted_map)

# Verify by composing F and F_inv
composition = inverted_map.compose(map_to_invert)
print("\n--- Composition of F_inv o F ---")
print(composition)
print("\n(Result should be close to the identity map [x, y])")

--- Original Map to Invert ---
TaylorMap with 2 components (input dim: 2):
--- Component 1 ---
         Coefficient  Order Exponents
0 1.000000000000e+00      1    (1, 0)
1 1.000000000000e-01      2    (0, 2)

--- Component 2 ---
          Coefficient  Order Exponents
0  1.000000000000e+00      1    (0, 1)
1 -1.000000000000e-01      2    (2, 0)



--- Inverted Map ---
TaylorMap with 2 components (input dim: 2):
--- Component 1 ---
          Coefficient  Order Exponents
0  1.000000000000e+00      1    (1, 0)
1 -1.000000000000e-01      2    (0, 2)
2 -2.000000000000e-02      3    (2, 1)
3 -1.000000000000e-03      4    (4, 0)
4  4.000000000000e-03      4    (1, 3)

--- Component 2 ---
          Coefficient  Order Exponents
0  1.000000000000e+00      1    (0, 1)
1  1.000000000000e-01      2    (2, 0)
2 -2.000000000000e-02      3    (1, 2)
3 -4.000000000000e-03      4    (3, 1)
4  1.000000000000e-03      4    (0, 4)



--- Composition of F_inv o F ---
TaylorMap with 2 components (input dim: 